In [25]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder

# Define relevant classes (e.g., for animals that interact with crops/livestock)
animal_classes = [
    'n01560419',  # Crow
    'n01817953',  # Pigeon
    'n01558993',  # European Starling
    'n01532829',  # Sparrow
    'n01843383',  # Hornbill
    'n01855672'   # Bee-eater
]

# Directory paths to your ImageNet dataset
train_dir = 'data/train'
val_dir = 'data/val'


# Define ImageDataGenerators
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

# Load and preprocess the data
train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse',
    classes=[str(cls) for cls in animal_classes]
)


val_data = datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),  # ImageNet standard size
    batch_size=32,
    class_mode='sparse',
    classes=[str(cls) for cls in animal_classes]
)


# Available classes
class_labels = list(train_data.class_indices.keys())

# Load the pre-trained model (VGG16) without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(animal_classes), activation='softmax')(x)  # Update to the number of classes

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data, epochs=10, validation_data=val_data)


Found 187 images belonging to 6 classes.
Found 18 images belonging to 6 classes.
Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 66s 10s/step - accuracy: 0.2154 - loss: 2.2277 - val_accuracy: 0.2778 - val_loss: 1.6025
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 65s 11s/step - accuracy: 0.2629 - loss: 1.8009 - val_accuracy: 0.4444 - val_loss: 1.4524
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 79s 10s/step - accuracy: 0.4578 - loss: 1.4303 - val_accuracy: 0.3889 - val_loss: 1.3317
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 61s 10s/step - accuracy: 0.5461 - loss: 1.2056 - val_accuracy: 0.6111 - val_loss: 1.1715
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 60s 10s/step - accuracy: 0.6281 - loss: 0.9885 - val_accuracy: 0.5556 - val_loss: 1.1726
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 61s 10s/step - accuracy: 0.6414 - loss: 0.8927 - val_accuracy: 0.6667 - val_loss: 1.0435
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 67s 11s/step - accuracy: 0.7073 - loss: 0.8535 - val_accuracy: 0.5556 - val_loss: 1.0848
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 60s 10s/

In [28]:
from tensorflow.keras.preprocessing import image
import numpy as np

def load_and_preprocess_image(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize
    return img_array

def predict_image_class(model, img_path, class_labels):
    # Load and preprocess the image
    img_array = load_and_preprocess_image(img_path)
    
    # Predict the class
    predictions = model.predict(img_array)
    class_index = np.argmax(predictions[0])
    predicted_label = class_labels[class_index]
    
    return predicted_label, predictions[0][class_index]

# Example usage
img_path = 'jack.png'
class_index, probability = predict_image_class(model, img_path, class_labels)
print(f"Predicted Class Index: {class_index}, Probability: {probability}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step
Predicted Class Index: n01532829, Probability: 0.40898844599723816


In [29]:
# RESNET
from tensorflow.keras.applications import ResNet50

# Load ResNet50 without the top layer
resnet_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in resnet_base_model.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(resnet_base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(animal_classes), activation='softmax')(x)

# Create the final model
resnet_model = Model(inputs=resnet_base_model.input, outputs=output)

# Compile the model
resnet_model.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
resnet_history = resnet_model.fit(train_data, epochs=10, validation_data=val_data)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 19s 0us/step
Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 43s 5s/step - accuracy: 0.1494 - loss: 3.8924 - val_accuracy: 0.1667 - val_loss: 2.7996
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 30s 5s/step - accuracy: 0.1742 - loss: 3.3511 - val_accuracy: 0.1667 - val_loss: 2.0430
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 30s 5s/step - accuracy: 0.1743 - loss: 2.5463 - val_accuracy: 0.2778 - val_loss: 2.1494
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 32s 5s/step - accuracy: 0.2619 - loss: 2.3002 - val_accuracy: 0.2222 - val_loss: 1.8209
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 33s 5s/step - accuracy: 0.1319 - loss: 2.0001 - val_accuracy: 0.2222 - val_loss: 1.7900
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 33s 5s/step - accuracy: 0.1578 - loss: 1.8586 - val_accuracy: 0.2778 - val_loss: 1.7592
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 40s 5s/step - accuracy: 0.2459 - loss: 1.7687 - val_accuracy: 0.2222 - val_loss: 1.7634
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 32s 5s/step - accuracy: 0.2170 - loss: 1.770

In [ ]:
# VGG19

from tensorflow.keras.applications import VGG19

# Load VGG19 without the top layer
vgg_base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in vgg_base_model.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(vgg_base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(animal_classes), activation='softmax')(x)

# Create the final model
vgg_model = Model(inputs=vgg_base_model.input, outputs=output)

# Compile the model
vgg_model.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
vgg_history = vgg_model.fit(train_data, epochs=10, validation_data=val_data)



Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 98s 13s/step - accuracy: 0.2967 - loss: 2.1639 - val_accuracy: 0.2222 - val_loss: 1.9228
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 85s 14s/step - accuracy: 0.2577 - loss: 1.8062 - val_accuracy: 0.4444 - val_loss: 1.6146
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 106s 19s/step - accuracy: 0.3296 - loss: 1.6114 - val_accuracy: 0.3889 - val_loss: 1.4056
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 84s 14s/step - accuracy: 0.4685 - loss: 1.2815 - val_accuracy: 0.5000 - val_loss: 1.2585
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 130s 21s/step - accuracy: 0.5743 - loss: 1.1093 - val_accuracy: 0.6111 - val_loss: 1.1603
Epoch 6/10
